In [1]:
import pandas as pd
import cv2
import torch
import numpy as np
import os.path
import random
import os


def seed_torch(seed=42):
    random.seed(seed)
    os.environ["PYTHONHASHSEED"] = str(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)
    torch.backends.cudnn.deterministic = True


seed_torch()
PATH = "/PATH TO DATA WITH No background/nbg"
PATH2 = "/PATH TO DATA/na/data"
lst = os.listdir(f"{PATH}/images")
lst2 = os.listdir(f"{PATH2}/images/all")

In [2]:
df = pd.read_csv(f"{PATH}/no_background.csv")

In [3]:
df.head()

,image_id,spacecraft_id,xmin,ymin,xmax,ymax
0,23c7f89d00006caee536a22e5807de95,5,994,596,1079,705
1,7e53cd9413f0333db10fab205c03bb0e,5,775,783,855,887
2,506da85290a45fdfdca3e07d19c07353,5,605,27,899,285
3,e3319f6372660a9ec4eea1a049f8ebfc,5,1061,946,1109,1024
4,b38e19187c0b4f37fd1731270d8e8f8a,5,835,525,940,635


In [4]:
len(df)

3750

In [ ]:
## make a directory with /crop/ name inside labels and images folders

In [35]:
from tqdm.notebook import tqdm


def convert_box(x1, y1, x2, y2, W, H):
    w = x2 - x1
    h = y2 - y1
    cx = x1 + w / 2
    cy = y1 + h / 2
    return [0, cx / W, cy / H, w / W, h / H]


def convert_box2(cx, cy, w, h):
    x1 = cx - (w // 2)
    y1 = cy - (h // 2)
    x2 = cx + (w // 2)
    y2 = cy + (h // 2)
    return [x1, y1, x2, y2]


def expand_region(box, w, h, l, m):
    noise = random.randint(5, 11)
    noise = noise / 10.0
    x1 = int(max(0, box[0] - noise * l))
    noise = random.randint(5, 11)
    noise = noise / 10.0
    y1 = int(max(0, box[1] - noise * m))
    noise = random.randint(5, 11)
    noise = noise / 10.0
    x2 = int(min(w, box[2] + noise * l))
    noise = random.randint(5, 11)
    noise = noise / 10.0
    y2 = int(min(h, box[3] + noise * m))
    return [x1, y1, x2, y2]


def shift_box(box, region):
    box[0] -= region[0]
    box[1] -= region[1]
    box[2] -= region[0]
    box[3] -= region[1]
    return box


for idx, i in tqdm(enumerate(lst2)):
    img = cv2.imread(f"{PATH2}/images/train/{i}")
    with open(f"{PATH2}/labels/train/{i.replace('.png', '.txt')}", "r") as f:
        txt = [float(x) for x in f.read().strip().split(" ")]
        txt[0] = int(txt[0])
    H, W, _ = img.shape
    cx = int(txt[1] * W)
    cy = int(txt[2] * H)
    w = int(txt[3] * W)
    h = int(txt[4] * H)
    box = convert_box2(cx, cy, w, h)
    region = expand_region(box, W, H, w, h)

    img2 = img[region[1] : region[3], region[0] : region[2], :]
    box = shift_box(box, region)
    nH, nW, _ = img2.shape
    bbox = convert_box(box[0], box[1], box[2], box[3], nW, nH)
    with open(f"{PATH2}/labels/crop/{i.replace('.png', '.txt')}", "w") as f:
        bbox = [str(k) for k in bbox]
        bbox = " ".join(bbox)
        f.write(bbox)
        f.write("\n")
    img2 = cv2.cvtColor(img2, cv2.COLOR_BGR2RGB)
    cv2.imwrite(f"{PATH2}/images/crop/{i}", img2)

0it [00:00, ?it/s]